# Daniel's Pre-Processing


In [48]:
import torch
print("CUDA available:", torch.cuda.is_available())
print("CUDA version:", torch.version.cuda)
print("Number of GPUs:", torch.cuda.device_count())
if torch.cuda.is_available():
    print("GPU name:", torch.cuda.get_device_name(0))

CUDA available: True
CUDA version: 12.9
Number of GPUs: 1
GPU name: NVIDIA GeForce MX450


Daniel's Branch Created


In [49]:
!pip install gensim


In [50]:
import pandas as pd
import numpy as np

In [51]:
train_df = pd.read_json('../Dataset/train.json')
test_df = pd.read_json('../Dataset/test.json')

In [52]:
train_df.head()

,premise,hypothesis,label
0,Pluto rotates once on its axis every 6.39 Eart...,Earth rotates on its axis once times in one day.,neutral
1,---Glenn =====================================...,Earth rotates on its axis once times in one day.,entails
2,geysers - periodic gush of hot water at the su...,The surface of the sun is much hotter than alm...,neutral
3,Facts: Liquid water droplets can be changed in...,Evaporation is responsible for changing liquid...,entails
4,"By comparison, the earth rotates on its axis o...",Earth rotates on its axis once times in one day.,entails


In [53]:
train_df.tail()

,premise,hypothesis,label
23083,"which is not only the motion of our bodies, bu...",Work is done only if a force is exerted in the...,neutral
23084,"The Red Star, that celestial curse whose eccen...",Red-shift refers to a shift toward red in the ...,neutral
23085,The lines in the spectrum of a luminous body s...,Red-shift refers to a shift toward red in the ...,entails
23086,The radial velocity of a star away from or tow...,Red-shift refers to a shift toward red in the ...,entails
23087,This expansion causes the light from distant s...,Red-shift refers to a shift toward red in the ...,entails


In [54]:
test_df.head()

,premise,hypothesis,label
0,Based on the list provided of the uses of subs...,"If a substance has a ph value greater than 7,t...",neutral
1,If one or two base pairs are change...,Invertebrates (and higher animals) can also be...,neutral
2,"At high temperatures, the solid dye converts i...",Gases and liquids become solids at low tempera...,neutral
3,Chapter 11 Gas and Kinetic Theory .,The behavior of ideal gases is explained by ki...,neutral
4,Both the continental crust and the oceanic cru...,Gabbro is a dark dense rock that can be found ...,neutral


# Data Pre-Processing


In [55]:
import nltk
if not nltk.download('punkt'):
    nltk.download('punkt')
    nltk.download('punkt_tab')
from nltk.tokenize import word_tokenize

if not nltk.download('stopwords'):
    nltk.download('stopwords')

from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

import re

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\DCCN9\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\DCCN9\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [56]:
# These are just common English contractions. We used it in Lab 5 before!
contraction_dict = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have",
                    "couldn't": "could not", "didn't": "did not",  "doesn't": "does not", "don't": "do not", "hadn't": "had not",
                    "hasn't": "has not", "haven't": "have not", "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did",
                    "how'd'y": "how do you", "how'll": "how will", "how's": "how is",  "I'd": "I would", "I'd've": "I would have",
                    "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would", "i'd've": "i would have",
                    "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would",
                    "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us",
                    "ma'am": "madam", "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have",
                    "must've": "must have", "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have",
                    "o'clock": "of the clock", "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not",
                    "shan't've": "shall not have", "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have",
                    "she's": "she is", "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have",
                    "so's": "so as", "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would",
                    "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would", "they'd've": "they would have",
                    "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have", "wasn't": "was not",
                    "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are", "we've": "we have",
                    "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are",  "what's": "what is", "what've": "what have",
                    "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is", "where've": "where have", "who'll": "who will",
                    "who'll've": "who will have", "who's": "who is", "who've": "who have", "why's": "why is", "why've": "why have", "will've": "will have",
                    "won't": "will not", "won't've": "will not have", "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have",
                    "y'all": "you all", "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have",
                    "you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have", "you're": "you are", "you've": "you have"}

In [57]:
# Stemmer and Lemmatizer
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.corpus import wordnet
from nltk import pos_tag, word_tokenize

if not nltk.download('wordnet'):
    nltk.download('wordnet')
    nltk.download('omw-1.4')
    nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\DCCN9\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [58]:
# Helper function for lemmatization with POS tagging
def get_wordnet_pos(treebank_tag):
    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN  # Default to noun if no match

In [59]:
# Allowed POS tags for filtering (example: nouns, verbs, adjectives, adverbs)
allowed_pos_tags = {'NN', 'NNS', 'NNP', 'NNPS',   # Nouns
                    'VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ',  # Verbs
                    'JJ', 'JJR', 'JJS',           # Adjectives
                    'RB', 'RBR', 'RBS'}           # Adverbs

In [60]:
# Filter the desired POS tags
def filter_tokens_by_pos(tokens):
    tagged_tokens = pos_tag(tokens)
    filtered = [word for word, tag in tagged_tokens if tag in allowed_pos_tags]
    return filtered

In [61]:
# Config
MIN_WORD_LENGTH = 3
MAX_WORD_LENGTH = 60 

In [62]:
def clean_dataset(dataset, MIN_WORD_LENGTH=3, MAX_WORD_LENGTH=50, method=None, pos_filter=False, stop_w=False):
    cleaned_premises = []
    cleaned_hypotheses = []
    cleaned_labels = []

    for index, row in dataset.iterrows():
        premise = row['premise']
        hypothesis = row['hypothesis']
        label = row['label']

        # Lowercase
        premise = premise.lower()
        hypothesis = hypothesis.lower()

        # Expand contractions
        for contraction, full_form in contraction_dict.items():
            premise = premise.replace(contraction, full_form)
            hypothesis = hypothesis.replace(contraction, full_form)

        # Remove punctuation/special chars
        premise = re.sub(r'[^\w\s]', '', premise)
        hypothesis = re.sub(r'[^\w\s]', '', hypothesis)

        # Normalize whitespace
        premise = re.sub(r'\s+', ' ', premise).strip()
        hypothesis = re.sub(r'\s+', ' ', hypothesis).strip()

        # Tokenization
        premise_tokens = word_tokenize(premise)
        hypothesis_tokens = word_tokenize(hypothesis)

        # Stemming/Lemmatization
        if method == 'stem':
            stemmer = PorterStemmer()
            premise_tokens = [stemmer.stem(word) for word in premise_tokens]
            hypothesis_tokens = [stemmer.stem(word) for word in hypothesis_tokens]
        elif method == 'lemmatize':
            lemmatizer = WordNetLemmatizer()
            premise_pos_tags = pos_tag(premise_tokens)
            hypothesis_pos_tags = pos_tag(hypothesis_tokens)
            premise_tokens = [lemmatizer.lemmatize(word, get_wordnet_pos(pos)) for word, pos in premise_pos_tags]
            hypothesis_tokens = [lemmatizer.lemmatize(word, get_wordnet_pos(pos)) for word, pos in hypothesis_pos_tags]

        # POS Filtering
        if pos_filter:
            premise_tokens = filter_tokens_by_pos(premise_tokens)
            hypothesis_tokens = filter_tokens_by_pos(hypothesis_tokens)

        # Remove stop words
        if stop_w:
            premise_tokens = [word for word in premise_tokens if word not in stop_words]
            hypothesis_tokens = [word for word in hypothesis_tokens if word not in stop_words]

        # Now check token length AFTER cleaning
        if (MIN_WORD_LENGTH <= len(premise_tokens) <= MAX_WORD_LENGTH and
            MIN_WORD_LENGTH <= len(hypothesis_tokens) <= MAX_WORD_LENGTH):
            cleaned_premises.append(premise_tokens)
            cleaned_hypotheses.append(hypothesis_tokens)
            cleaned_labels.append(label)
        # else: skip row

    # Build DataFrame from all cleaned token lists
    new_dataset = pd.DataFrame({
        'premise': cleaned_premises,
        'hypothesis': cleaned_hypotheses,
        'label': cleaned_labels
    })

    return new_dataset


In [63]:
clean_train_dataset = clean_dataset(train_df, stop_w=False, method=None, pos_filter=False)
clean_test_dataset = clean_dataset(test_df,stop_w=False, method=None, pos_filter= False )

clean_t_dataset = clean_train_dataset['premise'] + clean_train_dataset['hypothesis']

In [64]:
clean_train_dataset.head()

,premise,hypothesis,label
0,"[pluto, rotates, once, on, its, axis, every, 6...","[earth, rotates, on, its, axis, once, times, i...",neutral
1,"[glenn, once, per, day, the, earth, rotates, a...","[earth, rotates, on, its, axis, once, times, i...",entails
2,"[geysers, periodic, gush, of, hot, water, at, ...","[the, surface, of, the, sun, is, much, hotter,...",neutral
3,"[facts, liquid, water, droplets, can, be, chan...","[evaporation, is, responsible, for, changing, ...",entails
4,"[by, comparison, the, earth, rotates, on, its,...","[earth, rotates, on, its, axis, once, times, i...",entails


## Build dictionary of words and word2id


In [65]:
print(clean_train_dataset['premise'])

0        [pluto, rotates, once, on, its, axis, every, 6...
1        [glenn, once, per, day, the, earth, rotates, a...
2        [geysers, periodic, gush, of, hot, water, at, ...
3        [facts, liquid, water, droplets, can, be, chan...
4        [by, comparison, the, earth, rotates, on, its,...
                               ...                        
22900    [which, is, not, only, the, motion, of, our, b...
22901    [the, red, star, that, celestial, curse, whose...
22902    [the, lines, in, the, spectrum, of, a, luminou...
22903    [the, radial, velocity, of, a, star, away, fro...
22904    [this, expansion, causes, the, light, from, di...
Name: premise, Length: 22905, dtype: object


In [66]:
# Create a unique word list from the cleaned dataset
unique_words = set()
for sentence in clean_t_dataset: # Both Premise and Hypothesis 
    for word in sentence:
        unique_words.add(word)

print(f"Count of unique words: {len(unique_words)}")
#No lemmatization or stemming count = 22555
#Lemmatization count = 18986
#Stemming count = 15954


Count of unique words: 22134


In [67]:
unique_words_list = sorted(list(unique_words))

In [68]:
# No. of words with each method.

word2id = {w:i for i,w in enumerate(unique_words_list)}
id2word = {i:w for i,w in enumerate(unique_words_list)}


# Build Embedding Model using Gensim and FastText


In [69]:
from gensim.models import FastText

In [70]:
# Hyperparameters
batch_size = 1024
learn_rate = 0.001
embedding_size = 200
no_of_epochs = 8
window_size = 3
vocab_size = len(unique_words)


In [71]:
fast_text_model = FastText(clean_t_dataset, # Both premise and Hypothesis 
                           vector_size=embedding_size,
                           window=window_size,
                           sg=1,
                           epochs=no_of_epochs
                           )

In [72]:
fast_text_model.wv.most_similar('saturn', topn=10)

[('saturns', 0.9502410888671875),
 ('1980', 0.8152967691421509),
 ('neptune', 0.8126798868179321),
 ('1981', 0.7804783582687378),
 ('1980s', 0.7671456933021545),
 ('1979', 0.759753942489624),
 ('1989', 0.7521754503250122),
 ('uranus', 0.7484032511711121),
 ('spacecraft', 0.7481080889701843),
 ('voyager', 0.7424904108047485)]

# Transformer Model Implementation


## First we need to prepare the data for to input into the transformer.


Lets add some special tokens to my vocabularies to use as separation tokens [SEP] [PAD]


In [73]:
SPECIAL_TOKENS = ['[SEP]', '[PAD]']
for tok in SPECIAL_TOKENS:
    if tok not in word2id:
        idx = len(word2id)
        word2id[tok] = idx
        id2word[idx] = tok

sep_index = word2id['[SEP]']
pad_index = word2id['[PAD]']

In [74]:
# Hyperparameters for Transformer
#max_sequence_len = 64 

In [75]:
# Build a preparation data function for the transformer
def prepare_transformer_data(df,word2id,max_sequence_len):
    input_ids = []
    labels = []

    for _,row in df.iterrows():
        premise = row['premise']
        hypothesis = row['hypothesis']
        label = row['label']

        # Concatenate tokens + SEP + tokens
        tokens = premise + ['[SEP]'] + hypothesis

        # Convert to ids
        indices = [word2id.get(w, word2id['[PAD]']) for w in tokens]


        # PAD or truncate 
        if len(indices) < max_sequence_len:
            indices += [pad_index] * (max_sequence_len - len(indices))
        else:
            indices = indices[:max_sequence_len]

        input_ids.append(indices)
        labels.append(0 if label == 'neutral' else 1)

    return np.array(input_ids), np.array(labels)

In [76]:
x_train, y_train = prepare_transformer_data(clean_train_dataset, word2id, MAX_WORD_LENGTH)
x_test, y_test = prepare_transformer_data(clean_test_dataset, word2id, MAX_WORD_LENGTH)

In [77]:
word2id['[SEP]']

22134

In [78]:
print([id2word[idx] for idx in x_train[0]])
print(y_train[0])

['pluto', 'rotates', 'once', 'on', 'its', 'axis', 'every', '639', 'earth', 'days', '[SEP]', 'earth', 'rotates', 'on', 'its', 'axis', 'once', 'times', 'in', 'one', 'day', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]']
0


In [79]:
print(len(x_train[0]))

60


Now that we have the data as the transformer expects it, lets move to the transformer.
For the next steps i expect to build:

- Embedding Layer: Use my FastText with nn.Embeddings for the transformer
- Positional Encoding: Add the positional encoding to inject word order information.
- Transformer Encoder Layer
- Classification Head
- Forward Pass


In [84]:
#Hyperparameters Transformer 
BATCH_SIZE = 64
NUM_EPOCHS = 7
LR = 0.00005
NUM_CLASSES = 2
N_HEADS = 2
N_LAYERS = 2
DROPOUT = 0.2
WEIGHT_DECAY = 0.00005

In [85]:
import torch 
import torch.nn as nn
import math 

# Positional Encoding Class
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len):
        super().__init__()
        pe = torch.zeros(max_len, d_model) 
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0,d_model,2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)
        self.register_buffer('pe',pe)

    def forward(self,x):
        x = x + self.pe[:,:x.size(1),:]
        return x
    
# Transformer Class  
class TransformerClassifier(nn.Module):
    def __init__(self, embedding_matrix, num_classes):
        super().__init__()
        vocab_size, embed_dim = embedding_matrix.shape
        self.embedding = nn.Embedding.from_pretrained(torch.FloatTensor(embedding_matrix), freeze=False)
        self.pos_encoder = PositionalEncoding(embed_dim, max_len=MAX_WORD_LENGTH)

        encoder_layer = nn.TransformerEncoderLayer(d_model=embed_dim, nhead=N_HEADS, dropout=DROPOUT) # replace with hyperparameters in the future 
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=N_LAYERS) # Replace with hyperparameters 

        self.classifier = nn.Linear(embed_dim, num_classes)

    def forward(self, x):
        embedded = self.embedding(x) # [batch_size, seq_len, embed_dim]
        embedded = self.pos_encoder(embedded) # Add positional encoding 

        # Transformer expects [seq_len, batch_size, embed_dim]
        encoded = self.transformer_encoder(embedded.transpose(0,1))
        #Mean pool over seq_len: output shape [batch_size, embed_dim]
        pooled = encoded.mean(dim=0)

        logits = self.classifier(pooled)
        return logits 

Now lets build the embedding matrix from the FastText using the word2id


In [86]:
embedding_dim = fast_text_model.vector_size
vocab_size = len(word2id)

#Initialize the embedding matrix
embedding_matrix = np.zeros((vocab_size, embedding_dim))

for word, idx in word2id.items():
    if word in fast_text_model.wv:
        embedding_matrix[idx] = fast_text_model.wv[word]
    else:
        #Initialize unkown words
        embedding_matrix[idx] = np.random.normal(scale=0.6, size=(embedding_dim,))

# Convert the prepared input data 'x' to a PyTorch tensor
x_tensor = torch.LongTensor(x_train[:8]) # example mini-batch

#Instantiate your transformer Model
transformer_model = TransformerClassifier(embedding_matrix, num_classes=NUM_CLASSES)

# Run a forward pass
transformer_model.eval()
with torch.no_grad():
    logits = transformer_model(x_tensor)

print("Logits shape:", logits.shape)
print("Logits example output:",logits[0])


Logits shape: torch.Size([8, 2])
Logits example output: tensor([0.1752, 0.8858])


c:\Users\DCCN9\OneDrive - UWA\Data Science\Semester July-November 2025\CITS4012 - Natural Language Processing\Team Assignment 2 NLP\.venv\Lib\site-packages\torch\nn\modules\transformer.py:392: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(


For the last steps, lets define the optimizer and loss functin
Create the trainning loop
and build an evaluation function.


In [87]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset

# Convert the full datasets to PyTorch tensors
train_dataset = TensorDataset(torch.LongTensor(x_train), torch.LongTensor(y_train))
test_dataset = TensorDataset(torch.LongTensor(x_test), torch.LongTensor(y_test))

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE)

In [88]:
# Define loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(transformer_model.parameters(), lr=LR, weight_decay=WEIGHT_DECAY)

# Training function
def train_epoch(model, dataloader, criterion, optimizer, device):
    model.train()
    total_loss = 0
    correct = 0
    total = 0

    for inputs, labels in dataloader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item() * inputs.size(0)
        preds = outputs.argmax(dim=1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

    avg_loss = total_loss / total
    accuracy = correct / total
    return avg_loss, accuracy

# Evaluation function
def evaluate(model, dataloader, criterion, device):
    model.eval()
    total_loss = 0
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            total_loss += loss.item() * inputs.size(0)
            preds = outputs.argmax(dim=1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

    avg_loss = total_loss / total
    accuracy = correct / total
    return avg_loss, accuracy


Now we move to the trainning and evaluation


In [89]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
transformer_model.to(device)
print(device)
# Main training loop
for epoch in range(NUM_EPOCHS):
    train_loss, train_acc = train_epoch(transformer_model, train_loader, criterion, optimizer, device)
    val_loss, val_acc = evaluate(transformer_model, test_loader, criterion, device)
    print(f"Epoch {epoch+1}/{NUM_EPOCHS} - Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f} - Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}")

print("Training complete.")

cuda
Epoch 1/7 - Train Loss: 0.6505, Train Acc: 0.6337 - Val Loss: 0.6278, Val Acc: 0.6326
Epoch 2/7 - Train Loss: 0.5622, Train Acc: 0.7129 - Val Loss: 0.5546, Val Acc: 0.7292
Epoch 3/7 - Train Loss: 0.5064, Train Acc: 0.7530 - Val Loss: 0.6060, Val Acc: 0.7216
Epoch 4/7 - Train Loss: 0.4815, Train Acc: 0.7706 - Val Loss: 0.5593, Val Acc: 0.7363
Epoch 5/7 - Train Loss: 0.4492, Train Acc: 0.7876 - Val Loss: 0.6412, Val Acc: 0.7135
Epoch 6/7 - Train Loss: 0.4251, Train Acc: 0.8029 - Val Loss: 0.6987, Val Acc: 0.7074
Epoch 7/7 - Train Loss: 0.4022, Train Acc: 0.8164 - Val Loss: 0.6401, Val Acc: 0.7230
Training complete.


## Evaluation of the Model


In [90]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report

transformer_model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs = inputs.to(device)
        logits = transformer_model(inputs)
        preds = logits.argmax(dim=1).cpu().numpy()
        all_preds.extend(preds)
        all_labels.extend(labels.numpy())

print("Accuracy:", accuracy_score(all_labels, all_preds))
print("F1 Score:", f1_score(all_labels, all_preds))
print("Precision:", precision_score(all_labels, all_preds))
print("Recall:", recall_score(all_labels, all_preds))
print("-" * 40)
print(classification_report(all_labels, all_preds, target_names=["neutral","entails"]))

print('-' * 40)
print('Hyperparameters')
print('Batch Size',BATCH_SIZE)
print('Num of Epochs',NUM_EPOCHS)
print('Learning Rate',LR)
print('Num Classes',NUM_CLASSES)
print('Number of Heads',N_HEADS)
print('Numer of Layere',N_LAYERS)
print('Embedding Dimensions',embedding_dim)
print('Dropout',DROPOUT)
print('Weight Decay',WEIGHT_DECAY)

Accuracy: 0.7230113636363636
F1 Score: 0.5689019896831246
Precision: 0.7495145631067961
Recall: 0.4584323040380047
----------------------------------------
              precision    recall  f1-score   support

     neutral       0.71      0.90      0.80      1270
     entails       0.75      0.46      0.57       842

    accuracy                           0.72      2112
   macro avg       0.73      0.68      0.68      2112
weighted avg       0.73      0.72      0.71      2112

----------------------------------------
Hyperparameters
Batch Size 64
Num of Epochs 7
Learning Rate 5e-05
Num Classes 2
Number of Heads 2
Numer of Layere 2
Embedding Dimensions 200
Dropout 0.2
Weight Decay 5e-05
